In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import plotly.express as px 

In [2]:
import shutil

In [3]:
shutil.unpack_archive(
    "/content/drive/MyDrive/Data Car Resale Value Prediction (3).zip", 
    extract_dir="."
    )

In [4]:
data = pd.read_csv(
    "Data/autos.csv", encoding="Latin"
)

In [5]:
Xdata.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
cat_cols = list()
for colname in data.columns:
    subset = data.loc[:, colname]
    if subset.dtype == "object" or subset.dtype == "category":
        uniq_len = len(data.loc[:, colname].unique())
        print( colname, uniq_len)
        if uniq_len < 10:
            cat_cols.append(colname)

for catcol in cat_cols:
    data.loc[:,catcol] = data.loc[:,catcol].astype("category")


dateCrawled 280500
name 233531
seller 2
offerType 2
abtest 2
vehicleType 9
gearbox 3
model 252
fuelType 8
brand 40
notRepairedDamage 3
dateCreated 114
lastSeen 182806


In [7]:
data["dateCrawled"] = pd.to_datetime(data.dateCrawled)
data["lastSeen"] = pd.to_datetime(data.lastSeen)
data["monthOfRegistration"] = pd.to_numeric(data.monthOfRegistration, downcast="integer")
data["dateCreated"] = pd.to_datetime(data.dateCreated)

In [8]:
mini = 1900
current_year = datetime.date.today().year

# Seller contains only 1 type of data and so remove it.
data.drop("seller", axis=1, inplace=True)

# Offer contains Most of the data as Angebot and 12 data as Gesuch so remove it
data.drop("offerType", axis=1, inplace=True)

# Both DateCreated and DateCrawled are correlated
data.drop("dateCrawled", axis=1, inplace=True)

# REDUCE DATA RANGE BY YEAR OF REGISTRATION
data = data.loc[(data.yearOfRegistration > mini) & (data.yearOfRegistration < current_year)]

# nrOfPicture contains no information since it has only 1 data i.e) 0
data.drop("nrOfPictures", axis=1, inplace=True)



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
nrd_translate = {
    "ja" : "Yes",
    "nein" : "no",
}
data.notRepairedDamage.replace(nrd_translate, inplace=True)

gb_translate = {
    "automatik": "automatic",
    "manuell": "manual"
}

data.gearbox.replace(gb_translate, inplace=True)

fueltype_translate = {
    "benzin" : "petrol",
    "andere" : "other",
    "elektro" : "electro"
}
data.fuelType.replace(fueltype_translate, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [10]:
null_vals = data.isna().sum()
null_vals

name                       0
price                      0
abtest                     0
vehicleType            37687
yearOfRegistration         0
gearbox                20065
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType               33250
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
dtype: int64

In [11]:
data.reset_index(inplace=True, drop=True)
ft_by_model = data.groupby("model").fuelType.value_counts()

In [12]:
def impute_fueltype(model):
    if model is np.nan:
        return np.nan
    return ft_by_model[model].keys()[0]

tempo = data.loc[data.fuelType.isna(), "model"].apply(impute_fueltype).rename("fuelType")


In [13]:
ft_col = data.columns.get_loc("fuelType")
data.iloc[tempo.index, ft_col] = tempo

In [14]:
def impute_gearbox(model):
    if model is np.nan:
        return np.nan
    return gearbox_by_model[model].keys()[0]


In [15]:
gearbox_by_model = data.groupby("model").gearbox.value_counts()

In [16]:
tempo = data.loc[data.gearbox.isna(), "model"].apply(impute_gearbox).rename("gearbox")

In [17]:
gearbox_col = data.columns.get_loc("gearbox")
data.iloc[tempo.index, gearbox_col] = tempo

In [18]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType            37687
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
dtype: int64

In [19]:
vtype_by_model_brand = data.groupby(["model", "brand"]).vehicleType.value_counts()

In [20]:
data.groupby(["model", "brand"]).vehicleType.value_counts()

model   brand            
100     audi   limousine     278
               kombi         109
               coupe           2
               cabrio          1
               andere          0
                            ... 
zafira  opel   limousine      22
               kleinwagen     11
               suv             7
               cabrio          1
               coupe           1
Name: vehicleType, Length: 2384, dtype: int64

In [21]:
def impute_vehicletype(model):
    if model.model is np.nan or model.brand is np.nan:
        return np.nan

    return vtype_by_model_brand[model.model, model.brand].keys()[0]

In [22]:
vtype_by_model_brand["golf", "volkswagen"]

limousine     15337
kleinwagen     3929
kombi          3257
cabrio         2024
bus             795
coupe           435
andere          192
suv              25
Name: vehicleType, dtype: int64

In [23]:
tempo= data.loc[data.vehicleType.isna(), ["model", "brand"]].apply(impute_vehicletype, axis=1)

In [24]:
vtype_col = data.columns.get_loc("vehicleType")
data.iloc[tempo.index, vtype_col] = tempo

In [25]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType             6837
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
dtype: int64

In [26]:
data["model"].fillna(data.model.mode(), inplace=True)

In [27]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType             6837
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
dtype: int64

In [28]:
data.notRepairedDamage.value_counts()

no     263165
Yes     36277
Name: notRepairedDamage, dtype: int64

In [29]:
data.gearbox.fillna(data.gearbox.mode(), inplace=True)

In [30]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType             6837
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
dtype: int64

In [31]:
data["fuelType"] = data.fuelType.fillna(data.fuelType.mode())

In [32]:
data["vehicleType"] = data.vehicleType.astype("str").fillna(data.vehicleType.mode()).astype("category")

In [33]:
data["not_filled_repaired"] = data.notRepairedDamage.apply(lambda x: 1 if type(x) == float else 0)

In [34]:
data.notRepairedDamage.fillna(data.notRepairedDamage, inplace=True)

In [35]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType                0
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
not_filled_repaired        0
dtype: int64

In [36]:
data.head()

,name,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode,lastSeen,not_filled_repaired
0,Golf_3_1.6,480,test,limousine,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,70435,2016-04-07 03:16:57,1
1,A5_Sportback_2.7_Tdi,18300,test,coupe,2011,manual,190,NaN,125000,5,diesel,audi,Yes,2016-03-24,66954,2016-04-07 01:46:50,0
2,"Jeep_Grand_Cherokee_""Overland""",9800,test,suv,2004,automatic,163,grand,125000,8,diesel,jeep,NaN,2016-03-14,90480,2016-04-05 12:47:46,1
3,GOLF_4_1_4__3TÜRER,1500,test,kleinwagen,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,91074,2016-03-17 17:40:17,0
4,Skoda_Fabia_1.4_TDI_PD_Classic,3600,test,kleinwagen,2008,manual,69,fabia,90000,7,diesel,skoda,no,2016-03-31,60437,2016-04-06 10:17:21,0


In [37]:
data.isna().sum()

name                       0
price                      0
abtest                     0
vehicleType                0
yearOfRegistration         0
gearbox                 4161
powerPS                    0
model                  20404
kilometer                  0
monthOfRegistration        0
fuelType                7228
brand                      0
notRepairedDamage      71904
dateCreated                0
postalCode                 0
lastSeen                   0
not_filled_repaired        0
dtype: int64

In [41]:
from sklearn.model_selection import train_test_split 

target = "price"
X = data.drop(target, axis=1)
y = data.loc[:, target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
list(map(lambda x:x.shape, [X_train, X_test, y_train, y_test]))

[(297076, 16), (74270, 16), (297076,), (74270,)]